In [1]:
import os
os.chdir('..')
from dotenv import load_dotenv
from selenium import webdriver
from helper import set_filters_huurstunt
from helper_database import store_listing
from selenium.webdriver.common.by import By

In [2]:
load_dotenv()

HUURSTUNT_USERNAME = os.getenv('HUURSTUNT_USERNAME')
HUURSTUNT_PASSWORD = os.getenv('HUURSTUNT_PASSWORD')

chrome_options = webdriver.ChromeOptions()
chrome_options.debugger_address = "127.0.0.1:9222"
driver = webdriver.Chrome(options=chrome_options)


driver.get("https://www.huurstunt.nl/")
set_filters_huurstunt(driver, 100, 1500, 'Amsterdam')

In [3]:
listings = driver.find_elements(By.XPATH, '//div[contains(@class, "rental-card-wide") and contains(@class, "col-lg-12") and contains(@class, "col-md-12")]')
base_xpath = "/html/body/div[3]/div[2]/section[1]/div/div[2]/div[2]/div[2]/div/div[{}]/div/div/div/div/a"

for idx, listing in enumerate(listings, start=1):
    
    # Specific URL for each listing 
    xpath = base_xpath.format(idx)
    try:
        element = driver.find_element(By.XPATH, xpath)
        specific_url = element.get_attribute('href')
    except:
        continue  # or however you'd like to handle the absence of the URL


    parts = listing.text.split('\n')
    address = parts[1]
    price = parts[2]
    size = parts[4]
    
    print(address, price, size, specific_url)  # or process the data however you like
    print("-----------------------------")


1509 BS Zaandam (Zaanse Schans) € 945 34 m2 https://www.huurstunt.nl/studio/huren/in/zaandam/leeghwaterweg/PyvB2
-----------------------------
1505 DE Zaandam (Bomenbuurt) € 975 40 m2 https://www.huurstunt.nl/studio/huren/in/zaandam/zuiddijk/PyLkO
-----------------------------
1091 BD Amsterdam (Oosterparkbuurt Noordwest) € 1.350 32 m2 https://www.huurstunt.nl/studio/huren/in/amsterdam/tilanusstraat/PyAIC
-----------------------------
1071 JZ Amsterdam (Cornelis Schuytbuurt) € 1.500 29 m2 https://www.huurstunt.nl/studio/huren/in/amsterdam/banstraat/Pwr2n
-----------------------------
1062 JB Amsterdam (Delflandpleinbuurt West) € 1.390 36 m2 https://www.huurstunt.nl/studio/huren/in/amsterdam/voorburgstraat/Pu2IC
-----------------------------
1062 EV Amsterdam (Delflandpleinbuurt West) € 948 - https://www.huurstunt.nl/studio/huren/in/amsterdam/rijswijkstraat/PtxUw
-----------------------------
1115 TA Duivendrecht € 692 - https://www.huurstunt.nl/studio/huren/in/duivendrecht/lunaweg/PtxU

In [4]:
listings[0].text

'Leeghwaterweg\n1509 BS Zaandam (Zaanse Schans)\n€ 945\nPer 6 november te huur! Gemeubileerde studio welke onderdeel is van een woonboerderij aan de rand van Zaandam. De studio\n34 m2\n1 kamer\nGemeubileerd'

In [6]:
import sqlite3
from urllib.parse import urlparse

DATABASE_NAME = "listings.db"

def store_listing_(address, price=None, size=None, source_website_url=None):
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()

    # Extract until .com or .nl
    main_part = source_website_url.split(".com")[0] + ".com" if ".com" in source_website_url else source_website_url.split(".nl")[0] + ".nl"

    # Listing with the same specific_url already exists or not
    cursor.execute('SELECT * FROM listings WHERE specific_url = ?', (source_website_url,))
    existing_listing = cursor.fetchone()

    if not existing_listing:
        cursor.execute('''
        INSERT INTO listings (address, price, source_website, specific_url)
        VALUES (?, ?, ?, ?)
        ''', (address, price or "NaN", size or "NaN", main_part, source_website_url))

        conn.commit()

    conn.close()



store_listing_(address, price, size, specific_url)

OperationalError: 6 values for 4 columns

In [27]:
address

'1091 TT Amsterdam (Weesperzijde Midden/Zuid)'